Задача заключается в следующем: надо написать на Python функцию, которая получает на вход M, dT и файл с инцидентами, и вычисляет для каждого из N инцидентов количество инцидентов из того же файла, которые удовлетворяют следующим условиям:

1) предшествуют данному инциденту по времени, при этом разница по времени не превосходит dT;

2) имеют значения feature1 и feature2, совпадающие с соответствующими значениями данного инцидента.

Генерируем случайный набор инцидентов с N-строк, M-аргументов

In [ ]:
import pandas as pd
import numpy as np
from queue import Queue

In [ ]:
%%time

M = 100
N = 1000000

df = pd.DataFrame({'feature1':np.random.randint(M, size=(N,)),
                 'feature2':np.random.randint(M, size=(N,)),
                 'time':np.random.rand(N)
                 })
df.to_csv('incidents.csv', index_label='id') # =====

CPU times: user 3.38 s, sys: 54.5 ms, total: 3.43 s
Wall time: 3.44 s


Функция подсчета количества инцидентов за период dT 

In [ ]:
def process(input_file, output_file, M, dt):
    df = pd.read_csv(input_file, dtype={'feature1': int, 'feature2': int}, index_col='id')\
      .sort_values('time')
      
    features = (df['feature1'] + df['feature2'] * M).astype('int64').to_numpy()
    time_borders = (df['time'] - dt).to_numpy()
    times = df['time'].to_numpy()
    cache = [0] * M ** 2 
    q = Queue()
    counts = np.zeros(len(df))

    q.put(0)
    cache[features[0]] += 1

    for i in range(1, len(features)):            
        time_border = time_borders[i]
            
        while not q.empty():
            index = q.queue[0]
            
            if times[index] >= time_border:
              break

            cache[features[index]] -= 1
            q.get()
            
        q.put(i)
        counts[i] = cache[features[i]]
        cache[features[i]] += 1
                     
    df['count'] = counts
    df['count'] = df['count'].astype('int')
    df.sort_values('id').to_csv(output_file, columns=['count'])

Вычисляем. В ф-цию подаем входной/выходной файл, M и dt

In [ ]:
%%time
process('incidents.csv', 'out.csv', 100, 0.3)

CPU times: user 8.94 s, sys: 45 ms, total: 8.98 s
Wall time: 8.98 s


Если нам надо опустить переменную M при вызове функции, то убрать аргумент и внутри функции добавить:

M = max(df['feature1'].max(), df['feature2'].max()) + 1

Чтобы вызвать через командную строку надо ipynb сохранить как *.py и запустить с параметрами, например:

> python test_Skolkovo.py incidents.csv out.csv 100 0.3

In [ ]:
import sys

if __name__ == "__main__":
	if len(sys.argv) != 5:
		print ('Необходимо передать 4 параметра')
	else:
		process(sys.argv[1], sys.argv[2], int(sys.argv[3]), float(sys.argv[4]))
		print ('Обработка выполнена')

# Проверка на тестовом файле в 10 строк

Добавлено %%time, чтобы код не попадал при экспорте в py-файл

In [ ]:
%%time
df_in10 = pd.read_csv('inc10_t.csv', index_col='id')
df_in10.head(10)

,feature1,feature2,time
id,,,
0,1,0,0.206520
1,0,0,0.233725
2,0,1,0.760993
3,1,1,0.927770
4,1,0,0.569711
5,0,1,0.992246
6,0,0,0.593264
7,1,0,0.694181
8,1,1,0.823813


In [ ]:
%%time
process('inc10_t.csv', 'out10_t.csv', 2, 0.3)

In [ ]:
%%time
df10 = pd.read_csv('out10_t.csv')
df10.head(10)

,id,count
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0
5,5,2
6,6,0
7,7,1
8,8,0
9,9,1
